## Bibliotecas e Formatações

In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json
import requests
from tqdm import tqdm

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-10-21 19:49:30,711 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


## Próxima Rodada

In [2]:
# Mapeamento de clubes
def obter_mapeamento_clubes():
    response = requests.get("https://api.cartolafc.globo.com/clubes")
    clubes = response.json()
    return {int(clube_id): clube['nome'] for clube_id, clube in clubes.items()}

# Rodada atual
def obter_rodada_atual():
    url_status = "https://api.cartolafc.globo.com/mercado/status"
    return requests.get(url_status).json().get("rodada_atual", 1)

# Coletar todas as rodadas
def coletar_todas_rodadas():
    rodada_atual = obter_rodada_atual()
    clubes = obter_mapeamento_clubes()
    todas_partidas = []

    for rodada in tqdm(range(1, rodada_atual + 1), desc="🔄 Coletando rodadas"):
        url = f"https://api.cartolafc.globo.com/partidas/{rodada}"
        resp = requests.get(url).json()
        partidas = resp.get("partidas", [])

        for partida in partidas:
            data_partida = partida.get("partida_data", "")
            data_split = data_partida.split(" ")
            data = data_split[0] if len(data_split) > 0 else "indefinido"
            horario = data_split[1][:5] if len(data_split) > 1 else "indefinido"

            todas_partidas.append({
                "Rodada": rodada,
                "Data": data,
                "Horário": horario,
                "Local": partida.get("local", "desconhecido"),
                "Clube Casa": clubes.get(partida.get("clube_casa_id"), "Desconhecido"),
                "Placar Casa": partida.get("placar_oficial_mandante", ""),
                "Placar Visitante": partida.get("placar_oficial_visitante", ""),
                "Clube Visitante": clubes.get(partida.get("clube_visitante_id"), "Desconhecido")
            })

    return pd.DataFrame(todas_partidas)

# Executar
df_todas_rodadas = coletar_todas_rodadas()

# Salvar Excel
df_todas_rodadas.to_excel("rodadas_local_data.xlsx", index=False, engine="openpyxl")

# Salvar como JS
with open("rodadas_local_data.js", "w", encoding="utf-8") as f:
    f.write("const rodadasLocalData = ")
    json.dump(df_todas_rodadas.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    f.write(";")

print("✅ Arquivos salvos com sucesso.")


🔄 Coletando rodadas: 100%|██████████| 30/30 [00:08<00:00,  3.64it/s]


✅ Arquivos salvos com sucesso.


## Desempenhos dos Clubes

In [3]:
# Função para calcular o desempenho dos times
def calcular_desempenho_times(df_partidas):
    clubes_ids = pd.concat([df_partidas['Clube Casa'], df_partidas['Clube Visitante']]).unique()
    desempenho_times = {clube_id: {
        'Vitorias': 0, 'Empates': 0, 'Derrotas': 0, 'Pontos': 0, 'Jogos': 0, 
        'Gols Marcados': 0, 'Gols Sofridos': 0,
        'Gols Marcados Casa': 0, 'Gols Sofridos Casa': 0,
        'Gols Marcados Fora': 0, 'Gols Sofridos Fora': 0,
        'Vitorias Casa': 0, 'Empates Casa': 0, 'Derrotas Casa': 0,
        'Vitorias Fora': 0, 'Empates Fora': 0, 'Derrotas Fora': 0,
        'Jogos Casa': 0, 'Jogos Fora': 0  # Adicionando contagem de jogos em casa e fora
    } for clube_id in clubes_ids}

    for _, partida in df_partidas.iterrows():
        if pd.notnull(partida['Placar Casa']) and pd.notnull(partida['Placar Visitante']):
            desempenho_times[partida['Clube Casa']]['Jogos'] += 1
            desempenho_times[partida['Clube Visitante']]['Jogos'] += 1
            
            # Incrementar contagem de jogos em casa e fora
            desempenho_times[partida['Clube Casa']]['Jogos Casa'] += 1
            desempenho_times[partida['Clube Visitante']]['Jogos Fora'] += 1
            
            desempenho_times[partida['Clube Casa']]['Gols Marcados'] += partida['Placar Casa']
            desempenho_times[partida['Clube Casa']]['Gols Sofridos'] += partida['Placar Visitante']
            desempenho_times[partida['Clube Visitante']]['Gols Marcados'] += partida['Placar Visitante']
            desempenho_times[partida['Clube Visitante']]['Gols Sofridos'] += partida['Placar Casa']

            desempenho_times[partida['Clube Casa']]['Gols Marcados Casa'] += partida['Placar Casa']
            desempenho_times[partida['Clube Casa']]['Gols Sofridos Casa'] += partida['Placar Visitante']
            desempenho_times[partida['Clube Visitante']]['Gols Marcados Fora'] += partida['Placar Visitante']
            desempenho_times[partida['Clube Visitante']]['Gols Sofridos Fora'] += partida['Placar Casa']
            
            if partida['Placar Casa'] > partida['Placar Visitante']:
                desempenho_times[partida['Clube Casa']]['Vitorias'] += 1
                desempenho_times[partida['Clube Casa']]['Vitorias Casa'] += 1
                desempenho_times[partida['Clube Visitante']]['Derrotas'] += 1
                desempenho_times[partida['Clube Visitante']]['Derrotas Fora'] += 1
                desempenho_times[partida['Clube Casa']]['Pontos'] += 3
            elif partida['Placar Casa'] < partida['Placar Visitante']:
                desempenho_times[partida['Clube Visitante']]['Vitorias'] += 1
                desempenho_times[partida['Clube Visitante']]['Vitorias Fora'] += 1
                desempenho_times[partida['Clube Casa']]['Derrotas'] += 1
                desempenho_times[partida['Clube Casa']]['Derrotas Casa'] += 1
                desempenho_times[partida['Clube Visitante']]['Pontos'] += 3
            else:
                desempenho_times[partida['Clube Casa']]['Empates'] += 1
                desempenho_times[partida['Clube Casa']]['Empates Casa'] += 1
                desempenho_times[partida['Clube Visitante']]['Empates'] += 1
                desempenho_times[partida['Clube Visitante']]['Empates Fora'] += 1
                desempenho_times[partida['Clube Casa']]['Pontos'] += 1
                desempenho_times[partida['Clube Visitante']]['Pontos'] += 1

    df_desempenho = pd.DataFrame.from_dict(desempenho_times, orient='index').reset_index()
    df_desempenho.columns = ['Clube ID', 'Vitorias', 'Empates', 'Derrotas', 'Pontos', 'Jogos', 'Gols Marcados', 'Gols Sofridos',
                             'Gols Marcados Casa', 'Gols Sofridos Casa', 'Gols Marcados Fora', 'Gols Sofridos Fora',
                             'Vitorias Casa', 'Empates Casa', 'Derrotas Casa', 'Vitorias Fora', 'Empates Fora', 'Derrotas Fora',
                             'Jogos Casa', 'Jogos Fora']  # Adicionando colunas de contagem de jogos em casa e fora

    # Calcular Saldo de Gols Total
    df_desempenho['SG Total'] = df_desempenho['Gols Marcados'] - df_desempenho['Gols Sofridos']

    # Obter nomes dos clubes
    clubes = obter_mapeamento_clubes()
    df_desempenho['Nome Clube'] = df_desempenho['Clube ID'].map(clubes)

    colunas = ['Clube ID', 'Jogos', 'Pontos', 'Vitorias', 'Empates', 'Derrotas', 'SG Total', 
               'Gols Marcados', 'Gols Sofridos', 'Gols Marcados Casa', 'Gols Sofridos Casa', 'Gols Marcados Fora', 
               'Gols Sofridos Fora', 'Vitorias Casa', 'Empates Casa', 'Derrotas Casa', 'Vitorias Fora', 
               'Empates Fora', 'Derrotas Fora', 'Jogos Casa', 'Jogos Fora']
    
    df_desempenho = df_desempenho[colunas]
    df_desempenho = df_desempenho.sort_values(
        by=['Pontos', 'Vitorias', 'SG Total', 'Gols Marcados'],
        ascending=[False, False, False, False]
    ).reset_index(drop=True)

    # Converter colunas de gols e SG para inteiros
    colunas_gols = [
        'SG Total', 'Gols Marcados', 'Gols Sofridos',
        'Gols Marcados Casa', 'Gols Sofridos Casa',
        'Gols Marcados Fora', 'Gols Sofridos Fora'
    ]

    df_desempenho[colunas_gols] = df_desempenho[colunas_gols].astype(int)
    
    return df_desempenho

# Chamando a função de exemplo
df_desempenho = calcular_desempenho_times(df_todas_rodadas)
display(df_desempenho.head(10))

# Salvando arquivo em CSV
df_desempenho.to_csv("desempenho_times.csv", index=False)
print("✅ Arquivo CSV salvo como desempenho_times.csv")

# Salvando arquivo em JS
with open("desempenho_times.js", "w", encoding="utf-8") as f:
    f.write("const desempenhoTimes = ")
    json.dump(df_desempenho.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    f.write(";")

print("✅ Arquivo JS salvo como desempenho_times.js")

,Clube ID,Jogos,Pontos,Vitorias,Empates,Derrotas,SG Total,Gols Marcados,Gols Sofridos,Gols Marcados Casa,Gols Sofridos Casa,Gols Marcados Fora,Gols Sofridos Fora,Vitorias Casa,Empates Casa,Derrotas Casa,Vitorias Fora,Empates Fora,Derrotas Fora,Jogos Casa,Jogos Fora
0,PAL,28,61,19,4,5,27,53,26,33,11,20,15,11,2,2,8,2,3,15,13
1,FLA,28,61,18,7,3,41,56,15,36,6,20,9,10,5,0,8,2,3,15,13
2,CRU,29,56,16,8,5,21,42,21,27,11,15,10,11,2,2,5,6,3,15,14
3,MIR,29,52,14,10,5,20,50,30,34,13,16,17,10,5,0,4,5,5,15,14
4,BOT,29,46,13,7,9,13,39,26,24,16,15,10,7,4,3,6,3,6,14,15
5,BAH,28,46,13,7,8,7,39,32,25,10,14,22,10,3,1,3,4,7,14,14
6,FLU,28,41,12,5,11,0,35,35,18,10,17,25,9,1,3,3,4,8,13,15
7,VAS,29,39,11,6,12,5,46,41,25,18,21,23,7,4,4,4,2,8,15,14
8,SAO,29,38,10,8,11,-2,31,33,18,13,13,20,7,3,4,3,5,7,14,15
9,RBB,29,36,10,6,13,-10,34,44,19,19,15,25,6,4,4,4,2,9,14,15


✅ Arquivo CSV salvo como desempenho_times.csv
✅ Arquivo JS salvo como desempenho_times.js


## Análise de Eficiência/Desempenho dos Cluber em Casa e Fora

In [4]:
df_Vit_Emp_Der = df_desempenho.copy()

# Porcentagem de vitórias em casa
df_Vit_Emp_Der['% Vitorias Casa'] = df_Vit_Emp_Der['Vitorias Casa'] / df_Vit_Emp_Der['Jogos Casa'] * 100

# Porcentagem de vitórias fora
df_Vit_Emp_Der['% Vitorias Fora'] = df_Vit_Emp_Der['Vitorias Fora'] / df_Vit_Emp_Der['Jogos Fora'] * 100

# Porcentagem de empates em casa
df_Vit_Emp_Der['% Empates Casa'] = df_Vit_Emp_Der['Empates Casa'] / df_Vit_Emp_Der['Jogos Casa'] * 100

# Porcentagem de empates fora
df_Vit_Emp_Der['% Empates Fora'] = df_Vit_Emp_Der['Empates Fora'] / df_Vit_Emp_Der['Jogos Fora'] * 100

# Porcentagem de derrotas em casa
df_Vit_Emp_Der['% Derrotas Casa'] = df_Vit_Emp_Der['Derrotas Casa'] / df_Vit_Emp_Der['Jogos Casa'] * 100

# Porcentagem de derrotas fora
df_Vit_Emp_Der['% Derrotas Fora'] = df_Vit_Emp_Der['Derrotas Fora'] / df_Vit_Emp_Der['Jogos Fora'] * 100

# print(df_Vit_Emp_Der.columns)
colunas = ['Clube ID', #'Pontos', 'Jogos',
           '% Vitorias Casa', '% Empates Casa', '% Derrotas Casa',
           '% Vitorias Fora', '% Empates Fora', '% Derrotas Fora']

df_Vit_Emp_Der= df_Vit_Emp_Der[colunas]

# Exibir a tabela de "EFICIÊNCIA EM CASA E FORA" dos times
print("EFICIÊNCIA EM CASA E FORA")
display(df_Vit_Emp_Der.style.hide(axis='index').set_properties(**{'text-align': 'center'}).format(precision=1))


# Salvar em CSV
df_Vit_Emp_Der.to_csv("eficiencia_mandante_visitante.csv", index=False)

# Salvar em JS
with open("eficiencia_mandante_visitante.js", "w", encoding="utf-8") as f:
    f.write("const eficienciaMandanteVisitante = ")
    json.dump(df_Vit_Emp_Der.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    f.write(";")

print("✅ Arquivos salvos como eficiencia_mandante_visitante.csv e eficiencia_mandante_visitante.js")


EFICIÊNCIA EM CASA E FORA


Clube ID,% Vitorias Casa,% Empates Casa,% Derrotas Casa,% Vitorias Fora,% Empates Fora,% Derrotas Fora
PAL,73.3,13.3,13.3,61.5,15.4,23.1
FLA,66.7,33.3,0.0,61.5,15.4,23.1
CRU,73.3,13.3,13.3,35.7,42.9,21.4
MIR,66.7,33.3,0.0,28.6,35.7,35.7
BOT,50.0,28.6,21.4,40.0,20.0,40.0
BAH,71.4,21.4,7.1,21.4,28.6,50.0
FLU,69.2,7.7,23.1,20.0,26.7,53.3
VAS,46.7,26.7,26.7,28.6,14.3,57.1
SAO,50.0,21.4,28.6,20.0,33.3,46.7
RBB,42.9,28.6,28.6,26.7,13.3,60.0


✅ Arquivos salvos como eficiencia_mandante_visitante.csv e eficiencia_mandante_visitante.js


In [5]:
df_Media_Pontuacao_Geral = df_desempenho.copy()

# Média de pontos por jogo
df_Media_Pontuacao_Geral['Media Pontos por Jogo'] = df_Media_Pontuacao_Geral['Pontos'] / df_Media_Pontuacao_Geral['Jogos']

# Média de pontos por jogo em casa
df_Media_Pontuacao_Geral['Media Pontos por Jogo Casa'] = (df_Media_Pontuacao_Geral['Vitorias Casa'] * 3 + df_Media_Pontuacao_Geral['Empates Casa']) / df_Media_Pontuacao_Geral['Jogos Casa']

# Média de pontos por jogo fora
df_Media_Pontuacao_Geral['Media Pontos por Jogo Fora'] = (df_Media_Pontuacao_Geral['Vitorias Fora'] * 3 + df_Media_Pontuacao_Geral['Empates Fora']) / df_Media_Pontuacao_Geral['Jogos Fora']

# Total de pontos conquistados em casa
df_Media_Pontuacao_Geral['Total Pontos Casa'] = df_Media_Pontuacao_Geral['Vitorias Casa'] * 3 + df_Media_Pontuacao_Geral['Empates Casa']

# Total de pontos conquistados fora
df_Media_Pontuacao_Geral['Total Pontos Fora'] = df_Media_Pontuacao_Geral['Vitorias Fora'] * 3 + df_Media_Pontuacao_Geral['Empates Fora']

# print(df_Media_Pontuacao_Geral.columns)
colunas = ['Clube ID',
           'Media Pontos por Jogo', 'Media Pontos por Jogo Casa',
           'Media Pontos por Jogo Fora', 'Total Pontos Casa', 'Total Pontos Fora']

df_Media_Pontuacao_Geral = df_Media_Pontuacao_Geral[colunas]

# Exibir a tabela de "Desempenho Geral" dos times
print("Médias de Pontuações Geral")
display(df_Media_Pontuacao_Geral.style.hide(axis='index').set_properties(**{'text-align': 'center'}).format(precision=1))

# Salvar CSV
df_Media_Pontuacao_Geral.to_csv("media_pontos_clubes.csv", index=False)

# Salvar JS
with open("media_pontos_clubes.js", "w", encoding="utf-8") as f:
    f.write("const mediaPontosTimes = ")
    json.dump(df_Media_Pontuacao_Geral.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    f.write(";")

print("✅ Arquivos salvos com sucesso.")


Médias de Pontuações Geral


Clube ID,Media Pontos por Jogo,Media Pontos por Jogo Casa,Media Pontos por Jogo Fora,Total Pontos Casa,Total Pontos Fora
PAL,2.2,2.3,2.0,35,26
FLA,2.2,2.3,2.0,35,26
CRU,1.9,2.3,1.5,35,21
MIR,1.8,2.3,1.2,35,17
BOT,1.6,1.8,1.4,25,21
BAH,1.6,2.4,0.9,33,13
FLU,1.5,2.2,0.9,28,13
VAS,1.3,1.7,1.0,25,14
SAO,1.3,1.7,0.9,24,14
RBB,1.2,1.6,0.9,22,14


✅ Arquivos salvos com sucesso.


## Análise Média de Gols

In [6]:
df_gols = df_desempenho.copy()

# Média de gols marcados por jogo em casa
df_gols['Media Gols Marcados Casa'] = df_gols['Gols Marcados Casa'] / df_gols['Jogos Casa']

# Média de gols marcados por jogo fora
df_gols['Media Gols Marcados Fora'] = df_gols['Gols Marcados Fora'] / df_gols['Jogos Fora']

# Média de gols sofridos por jogo em casa
df_gols['Media Gols Sofridos Casa'] = df_gols['Gols Sofridos Casa'] / df_gols['Jogos Casa']

# Média de gols sofridos por jogo fora
df_gols['Media Gols Sofridos Fora'] = df_gols['Gols Sofridos Fora'] / df_gols['Jogos Fora']

# Saldo de gols em casa
df_gols['Saldo Gols Casa'] = df_gols['Gols Marcados Casa'] - df_gols['Gols Sofridos Casa']

# Saldo de gols fora
df_gols['Saldo Gols Fora'] = df_gols['Gols Marcados Fora'] - df_gols['Gols Sofridos Fora']

# print(df_gols.columns)

colunas = ['Clube ID',
       'Media Gols Marcados Casa', 'Media Gols Marcados Fora',
       'Media Gols Sofridos Casa', 'Media Gols Sofridos Fora',
       'Saldo Gols Casa', 'Saldo Gols Fora']

df_gols = df_gols[colunas]

# Exibir a tabela de "MÉDIA DE GOLS" dos times
print("MÉDIA DE GOLS")
display(df_gols.style.hide(axis='index').set_properties(**{'text-align': 'center'}).format(precision=1))


# Salvar em CSV
df_gols.to_csv("media_gols_clubes.csv", index=False)

# Salvar como JS
with open("media_gols_clubes.js", "w", encoding="utf-8") as f:
    f.write("const mediaGolsClubes = ")
    json.dump(df_gols.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    f.write(";")

print("✅ Arquivos salvos como media_gols_clubes.csv e media_gols_clubes.js")


MÉDIA DE GOLS


Clube ID,Media Gols Marcados Casa,Media Gols Marcados Fora,Media Gols Sofridos Casa,Media Gols Sofridos Fora,Saldo Gols Casa,Saldo Gols Fora
PAL,2.2,1.5,0.7,1.2,22,5
FLA,2.4,1.5,0.4,0.7,30,11
CRU,1.8,1.1,0.7,0.7,16,5
MIR,2.3,1.1,0.9,1.2,21,-1
BOT,1.7,1.0,1.1,0.7,8,5
BAH,1.8,1.0,0.7,1.6,15,-8
FLU,1.4,1.1,0.8,1.7,8,-8
VAS,1.7,1.5,1.2,1.6,7,-2
SAO,1.3,0.9,0.9,1.3,5,-7
RBB,1.4,1.0,1.4,1.7,0,-10


✅ Arquivos salvos como media_gols_clubes.csv e media_gols_clubes.js


In [7]:
# === Funções auxiliares ===

def obter_rodada_atual():
    url_status = "https://api.cartolafc.globo.com/mercado/status"
    return requests.get(url_status).json().get("rodada_atual", 1)

def obter_mapeamento_clubes():
    response = requests.get("https://api.cartolafc.globo.com/clubes")
    clubes = response.json()
    return {int(clube_id): clube['nome'] for clube_id, clube in clubes.items()}

def obter_dados_partidas_rodadas(rodadas=None):
    if rodadas is None:
        rodada_atual = obter_rodada_atual()
        rodadas = range(1, rodada_atual + 1)

    dados = []
    for rodada in rodadas:
        url = f"https://api.cartolafc.globo.com/partidas/{rodada}"
        resp = requests.get(url).json()
        for partida in resp.get("partidas", []):
            dados.append({
                "Rodada": rodada,
                "Clube Casa": partida["clube_casa_id"],
                "Clube Visitante": partida["clube_visitante_id"],
                "Placar Casa": partida.get("placar_oficial_mandante", None),
                "Placar Visitante": partida.get("placar_oficial_visitante", None),
            })
    return pd.DataFrame(dados)

def calcular_desempenho_times(df_partidas):
    clubes_ids = pd.concat([df_partidas['Clube Casa'], df_partidas['Clube Visitante']]).unique()
    desempenho = {cid: {
        'Vitorias': 0, 'Empates': 0, 'Derrotas': 0, 'Pontos': 0, 'Jogos': 0,
        'Gols Marcados': 0, 'Gols Sofridos': 0,
        'Gols Marcados Casa': 0, 'Gols Sofridos Casa': 0,
        'Gols Marcados Fora': 0, 'Gols Sofridos Fora': 0,
        'Vitorias Casa': 0, 'Empates Casa': 0, 'Derrotas Casa': 0,
        'Vitorias Fora': 0, 'Empates Fora': 0, 'Derrotas Fora': 0,
        'Jogos Casa': 0, 'Jogos Fora': 0
    } for cid in clubes_ids}

    for _, row in df_partidas.iterrows():
        casa, fora = row['Clube Casa'], row['Clube Visitante']
        g_casa, g_fora = row['Placar Casa'], row['Placar Visitante']
        if pd.notnull(g_casa) and pd.notnull(g_fora):
            desempenho[casa]['Jogos'] += 1
            desempenho[fora]['Jogos'] += 1
            desempenho[casa]['Jogos Casa'] += 1
            desempenho[fora]['Jogos Fora'] += 1

            desempenho[casa]['Gols Marcados'] += g_casa
            desempenho[casa]['Gols Sofridos'] += g_fora
            desempenho[fora]['Gols Marcados'] += g_fora
            desempenho[fora]['Gols Sofridos'] += g_casa

            desempenho[casa]['Gols Marcados Casa'] += g_casa
            desempenho[casa]['Gols Sofridos Casa'] += g_fora
            desempenho[fora]['Gols Marcados Fora'] += g_fora
            desempenho[fora]['Gols Sofridos Fora'] += g_casa

            if g_casa > g_fora:
                desempenho[casa]['Vitorias'] += 1
                desempenho[casa]['Vitorias Casa'] += 1
                desempenho[fora]['Derrotas'] += 1
                desempenho[fora]['Derrotas Fora'] += 1
                desempenho[casa]['Pontos'] += 3
            elif g_fora > g_casa:
                desempenho[fora]['Vitorias'] += 1
                desempenho[fora]['Vitorias Fora'] += 1
                desempenho[casa]['Derrotas'] += 1
                desempenho[casa]['Derrotas Casa'] += 1
                desempenho[fora]['Pontos'] += 3
            else:
                desempenho[casa]['Empates'] += 1
                desempenho[fora]['Empates'] += 1
                desempenho[casa]['Empates Casa'] += 1
                desempenho[fora]['Empates Fora'] += 1
                desempenho[casa]['Pontos'] += 1
                desempenho[fora]['Pontos'] += 1

    df = pd.DataFrame.from_dict(desempenho, orient="index").reset_index()
    df.rename(columns={'index': 'Clube ID'}, inplace=True)
    df['SG Total'] = df['Gols Marcados'] - df['Gols Sofridos']
    df['Clube ID'] = df['Clube ID'].map(obter_mapeamento_clubes())
    return df

def calcular_pesos_times(df):
    df = df.sort_values(by="Pontos", ascending=False).reset_index(drop=True)

    df['Grupo'] = pd.cut(df.index, bins=[-1, 3, 7, 11, 15, 19], labels=[50, 40, 30, 20, 10])
    df['Grupo'] = df['Grupo'].astype(int)

    tradicao = {'FLA': 100, 'PAL': 90, 'BOT': 80, 'SAO': 70, 'CAM': 80, 'CRU': 60, 'INT': 90, 'COR': 80,
                'GRE': 60, 'FLU': 70, 'FOR': 70, 'BAH': 80, 'VAS': 70, 'RBB': 70, 'JUV': 70, 'VIT': 70,
                'SAN': 75, 'MIR': 60, 'SPT': 60, 'CEA': 60}
    df['Pontos Tradição'] = df['Clube ID'].map(tradicao)

    df['SG Casa'] = df['Gols Marcados Casa'] - df['Gols Sofridos Casa']
    df['SG Fora'] = df['Gols Marcados Fora'] - df['Gols Sofridos Fora']

    df['Peso Casa'] = (
        df['Grupo'] +
        df['Pontos Tradição'] * 1.2 +
        df['SG Total'] * 0.2 +
        df['SG Casa'] * 0.2 +
        df['Vitorias Casa'] * 2 -
        df['Derrotas Casa'] * 1
    )

    df['Peso Fora'] = (
        df['Grupo'] +
        df['Pontos Tradição'] * 0.8 +
        df['SG Total'] * 0.2 +
        df['SG Fora'] * 0.2 +
        df['Vitorias Fora'] * 2 -
        df['Derrotas Fora'] * 1
    )

    return df

# === Execução ===

rodadas = list(range(1, obter_rodada_atual() + 1))
df_partidas = obter_dados_partidas_rodadas(rodadas)
df_desempenho = calcular_desempenho_times(df_partidas)
df_pesos = calcular_pesos_times(df_desempenho)

# Selecionar colunas principais
df_final = df_pesos[['Clube ID', 'Jogos', 'Pontos', 'Gols Marcados', 'Gols Sofridos', 'SG Total',
                     'Peso Casa', 'Peso Fora']].copy()

# === Salvar arquivos ===

df_final.to_excel("pesos_times.xlsx", index=False, engine='openpyxl')
df_final.to_csv("pesos_times.csv", index=False)

with open("pesos_times.js", "w", encoding="utf-8") as f:
    f.write("const pesosTimes = ")
    json.dump(df_final.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    f.write(";")

print("✅ Arquivos salvos com sucesso!")



✅ Arquivos salvos com sucesso!


In [8]:
display(df_final.head(10))

,Clube ID,Jogos,Pontos,Gols Marcados,Gols Sofridos,SG Total,Peso Casa,Peso Fora
0,FLA,28,61,56.00,15.00,41.00,204.20,153.40
1,PAL,28,61,53.00,26.00,27.00,187.80,141.40
2,CRU,29,56,42.00,21.00,21.00,149.40,110.20
3,MIR,29,52,50.00,30.00,20.00,150.20,104.80
4,BOT,29,46,39.00,26.00,13.00,151.20,113.60
5,BAH,28,46,39.00,32.00,7.00,159.40,102.80
6,FLU,28,41,35.00,35.00,0.00,140.60,92.40
7,VAS,29,39,46.00,41.00,5.00,136.40,96.60
8,SAO,29,38,31.00,33.00,-2.00,124.60,83.20
9,COR,29,36,31.00,35.00,-4.00,134.40,90.20


### Buscando Scouts Gerais

In [9]:
def obter_dados_rodadas():
    # Verifica a rodada atual
    status_url = "https://api.cartola.globo.com/mercado/status"
    status_response = requests.get(status_url)
    rodada_atual = status_response.json().get('rodada_atual', 0)

    print(f"Rodada atual detectada: {rodada_atual}")
    
    todos_dados = []

    for rodada in tqdm(range(1, rodada_atual + 1), desc="Buscando rodadas"):
        try:
            # Scouts da rodada
            url = f"https://api.cartola.globo.com/atletas/pontuados/{rodada}"
            resp = requests.get(url)
            data = resp.json()

            atletas = data.get('atletas', {})
            clubes = data.get('clubes', {})
            posicoes = data.get('posicoes', {})

            # Jogos da rodada
            url_partidas = f"https://api.cartola.globo.com/partidas/{rodada}"
            partidas = requests.get(url_partidas).json().get("partidas", [])

            dict_mando = {}
            for jogo in partidas:
                dict_mando[jogo["clube_casa_id"]] = "mandante"
                dict_mando[jogo["clube_visitante_id"]] = "visitante"

            # Monta dataframe por rodada
            for atleta_id, info in atletas.items():
                atleta = {
                    "Rodada": rodada,
                    "ID": atleta_id,
                    "Nome": info["apelido"],
                    "Clube": clubes[str(info["clube_id"])]["nome"],
                    "Posição": posicoes[str(info["posicao_id"])]["nome"],
                    "Mandante_Visitante": dict_mando.get(info["clube_id"], "desconhecido"),
                    "Pontuação": info["pontuacao"]
                }

                scouts = info.get("scout", {})
                for scout, valor in scouts.items():
                    atleta[scout] = valor

                todos_dados.append(atleta)

        except Exception as e:
            print(f"❌ Erro na rodada {rodada}: {e}")

    df = pd.DataFrame(todos_dados)
    return df

# Executa e salva
df_scouts = obter_dados_rodadas()
df_scouts.to_csv("scouts_rodada_detalhado.csv", index=False)
print("✅ Dados salvos em 'scouts_rodada_detalhado.csv'")


Rodada atual detectada: 30


Buscando rodadas:  13%|█▎        | 4/30 [00:00<00:05,  4.73it/s]

❌ Erro na rodada 4: 'NoneType' object has no attribute 'items'


Buscando rodadas:  23%|██▎       | 7/30 [00:01<00:04,  4.63it/s]

❌ Erro na rodada 7: 'NoneType' object has no attribute 'items'


...

❌ Erro na rodada 16: 'NoneType' object has no attribute 'items'


Buscando rodadas:  87%|████████▋ | 26/30 [00:05<00:00,  4.28it/s]

❌ Erro na rodada 26: 'NoneType' object has no attribute 'items'


Buscando rodadas: 100%|██████████| 30/30 [00:06<00:00,  4.48it/s]

✅ Dados salvos em 'scouts_rodada_detalhado.csv'


In [10]:
import requests
import pandas as pd
from tqdm import tqdm

def coletar_detalhes_rodadas():
    # 1. Verifica a rodada atual
    status_url = "https://api.cartola.globo.com/mercado/status"
    status = requests.get(status_url).json()
    rodada_atual = status.get("rodada_atual", 0)

    print(f"📦 Rodada atual detectada: {rodada_atual}")

    todos_dados = []

    for rodada in tqdm(range(1, rodada_atual + 1), desc="🔄 Coletando rodadas"):
        try:
            # 2. Dados de pontuação
            url_pontuados = f"https://api.cartola.globo.com/atletas/pontuados/{rodada}"
            resp = requests.get(url_pontuados)
            data = resp.json()

            atletas = data.get('atletas', {})
            clubes = data.get('clubes', {})
            posicoes = data.get('posicoes', {})

            # 3. Dados das partidas da rodada
            url_partidas = f"https://api.cartola.globo.com/partidas/{rodada}"
            partidas = requests.get(url_partidas).json().get("partidas", [])

            dict_mando = {}
            dict_adversario = {}

            for jogo in partidas:
                casa_id = jogo["clube_casa_id"]
                visita_id = jogo["clube_visitante_id"]
                dict_mando[casa_id] = "mandante"
                dict_mando[visita_id] = "visitante"
                dict_adversario[casa_id] = visita_id
                dict_adversario[visita_id] = casa_id

            # 4. Monta os dados por jogador
            for atleta_id, info in atletas.items():
                clube_id = info["clube_id"]
                pos_id = info["posicao_id"]
                status_id = info.get("status_id", 0)

                jogador = {
                    "Rodada": rodada,
                    "ID": atleta_id,
                    "Nome": info["apelido"],
                    "Clube": clubes[str(clube_id)]["nome"],
                    "Posição": posicoes[str(pos_id)]["nome"],
                    "Mandante_Visitante": dict_mando.get(clube_id, "desconhecido"),
                    "Adversário": clubes.get(str(dict_adversario.get(clube_id, "")), {}).get("nome", "Desconhecido"),
                    "Pontuação": info["pontuacao"],                    
                    "Status_ID": status_id,                    
                }

                # Scouts
                for scout, valor in (info.get("scout") or {}).items():
                    jogador[scout] = valor

                todos_dados.append(jogador)

        except Exception as e:
            print(f"❌ Erro na rodada {rodada}: {e}")

    df = pd.DataFrame(todos_dados)
    return df

# Executa a coleta e salva os dados
df_detalhado = coletar_detalhes_rodadas()

scout_cols = ['FD', 'FS', 'CA', 'FC', 'DS', 'FF', 'SG', 'A', 'G', 'I', 'DE', 'GS', 'PC', 'DP', 'CV', 'FT', 'PP', 'PS', 'GC', 'V']

# Preenche os NaNs com 0
df_detalhado[scout_cols] = df_detalhado[scout_cols].fillna(0)

# Converte para inteiro (int)
df_detalhado[scout_cols] = df_detalhado[scout_cols].astype("int32")


df_detalhado.to_csv("scouts_detalhado.csv", index=False)
print("✅ Arquivo salvo como scouts_detalhado.csv")


📦 Rodada atual detectada: 30


🔄 Coletando rodadas: 100%|██████████| 30/30 [00:06<00:00,  4.45it/s]

✅ Arquivo salvo como scouts_detalhado.csv


In [11]:
display(df_detalhado.tail(10))

,Rodada,ID,Nome,Clube,Posição,Mandante_Visitante,Adversário,Pontuação,Status_ID,FD,FS,CA,FC,DS,FF,SG,A,G,I,DE,GS,V,PC,DP,CV,FT,PP,PS,GC
9059,29,98720,Angileri,COR,Lateral,mandante,CAM,5.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9060,29,98873,Samuel Lino,FLA,Atacante,mandante,PAL,1.50,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9061,29,99032,Lourenço,CEA,Meia,mandante,BOT,0.80,0,0,0,1,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9062,29,99119,Lucas Braga,VIT,Atacante,visitante,SAN,3.80,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
9063,29,99198,Vegetti,VAS,Atacante,mandante,FLU,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9064,29,99267,Pochettino,FOR,Meia,visitante,CRU,4.40,0,1,0,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9065,29,99366,Luis Zubeldía,FLU,Técnico,visitante,VAS,3.18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9066,29,99440,Baralhas,VIT,Meia,visitante,SAN,4.40,0,1,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9067,29,99552,Rossi,FLA,Goleiro,mandante,PAL,6.30,0,0,1,0,0,0,0,0,0,0,0,6,2,0,0,0,0,0,0,0,0
9068,29,99889,Alan Franco,SAO,Zagueiro,visitante,MIR,0.50,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


### Buscando pontuação em tempo real na API

In [12]:
import requests
import pandas as pd

# Consulta o status do mercado
status_url = "https://api.cartola.globo.com/mercado/status"
status_response = requests.get(status_url)
status_data = status_response.json()

mercado_status = status_data.get('status_mercado', None)
print("Status do Mercado:", mercado_status)

if mercado_status == 1:
    print("O mercado está aberto para escalações. A rodada ainda não começou, então as pontuações parciais não estarão disponíveis.")

elif mercado_status == 2:
    print("A rodada está em andamento. Realizando consulta de pontuações parciais...")

    url = "https://api.cartola.globo.com/atletas/pontuados"
    response = requests.get(url)

    if response.status_code == 200 and response.text.strip():
        try:
            data = response.json()
            jogadores = data.get('atletas', {})
            clubes = data.get('clubes', {})
            posicoes = data.get('posicoes', {})

            lista_jogadores = []
            for atleta_id, info in jogadores.items():
                clube_nome = clubes[str(info['clube_id'])]['nome']
                posicao_nome = posicoes[str(info['posicao_id'])]['nome']
                lista_jogadores.append({
                    'ID': atleta_id,
                    'Nome': info['apelido'],
                    'Clube': clube_nome,
                    'Posição': posicao_nome,
                    'Pontuação': info['pontuacao']
                })

            df_parciais = pd.DataFrame(lista_jogadores)
            print(df_parciais.head())

        except Exception as e:
            print("Erro ao interpretar JSON:", e)
    else:
        print("⚠️ Nenhum dado de pontuação parcial disponível.")

elif mercado_status == 3:
    print("O mercado está atualizando os dados pós-rodada.")
else:
    print("Status do mercado desconhecido. Verifique se a API está operando corretamente.")


Status do Mercado: 1
O mercado está aberto para escalações. A rodada ainda não começou, então as pontuações parciais não estarão disponíveis.


In [13]:
import requests
import pandas as pd

def buscar_id_por_nome(nome_time):
    url_busca = f"https://api.cartola.globo.com/times?q={nome_time}"
    resp = requests.get(url_busca)
    if resp.status_code == 200:
        times = resp.json()
        if times:
            time_encontrado = times[0]  # pega o primeiro resultado
            print(f"🔍 Time encontrado: {time_encontrado['nome']} (ID: {time_encontrado['time_id']})")
            return time_encontrado['time_id']
        else:
            print("❌ Nenhum time encontrado com esse nome.")
            return None
    else:
        print("❌ Erro ao buscar time pelo nome.")
        return None

def obter_pontuacao_parcial_time(identificador):
    # Verifica se o identificador é numérico (ID) ou nome (string)
    if isinstance(identificador, int) or identificador.isdigit():
        time_id = int(identificador)
    else:
        time_id = buscar_id_por_nome(identificador)
        if time_id is None:
            return

    # Passo 1: Consulta os jogadores escalados
    url_escalacao = f"https://api.cartola.globo.com/time/id/{time_id}"
    resp_time = requests.get(url_escalacao)
    
    if resp_time.status_code != 200:
        print("❌ Erro ao buscar dados do time.")
        return
    
    dados_time = resp_time.json()
    atletas_escalação = dados_time.get("atletas", [])

    if not atletas_escalação:
        print("❌ Nenhum jogador escalado ou mercado já fechado.")
        return

    # Passo 2: Consulta as pontuações parciais
    url_parciais = "https://api.cartola.globo.com/atletas/pontuados"
    resp_parciais = requests.get(url_parciais)

    if resp_parciais.status_code != 200:
        print("❌ Erro ao buscar pontuações parciais.")
        return

    data_parciais = resp_parciais.json()
    pontuados = data_parciais.get("atletas", {})
    clubes = data_parciais.get("clubes", {})
    posicoes = data_parciais.get("posicoes", {})

    # Passo 3: Monta DataFrame com as pontuações dos jogadores escalados
    lista = []
    for atleta in atletas_escalação:
        atleta_id = atleta['atleta_id']
        if str(atleta_id) in pontuados:
            dados = pontuados[str(atleta_id)]
            nome = dados['apelido']
            pontuacao = dados['pontuacao']
            clube = clubes[str(dados['clube_id'])]['nome']
            posicao = posicoes[str(dados['posicao_id'])]['nome']
            lista.append({
                'Nome': nome,
                'Clube': clube,
                'Posição': posicao,
                'Pontuação': pontuacao
            })

    df = pd.DataFrame(lista)
    print(df)
    print(f"\n➡️ Pontuação Parcial do Time: {df['Pontuação'].sum():.2f} pontos")

# 🟢 Exemplo de uso:
# Com nome
# obter_pontuacao_parcial_time("FBC Colorado")

# Com ID
obter_pontuacao_parcial_time("3708025")


❌ Erro ao buscar pontuações parciais.


In [33]:
import pandas as pd
import requests
from time import sleep

# === 1. Lê o arquivo com os times da liga ===
arquivo = "links_times_cartola_liga_serie_A.xlsx"

df_times = pd.read_excel(arquivo)

display(df_times.head())

# Verifica se as colunas esperadas existem
colunas_esperadas = ['Nome do Time', 'ID do Time']
if not all(col in df_times.columns for col in colunas_esperadas):
    raise ValueError(f"O arquivo deve conter as colunas: {colunas_esperadas}")

# === 2. Consulta pontuações parciais ===
print("🔄 Consultando pontuações parciais...")
url_parciais = "https://api.cartola.globo.com/atletas/pontuados"
resp_parciais = requests.get(url_parciais)
if resp_parciais.status_code != 200:
    raise Exception("❌ Erro ao buscar pontuações parciais.")

data_parciais = resp_parciais.json()
pontuados = data_parciais.get("atletas", {})
clubes = data_parciais.get("clubes", {})
posicoes = data_parciais.get("posicoes", {})

# === 3. Loop para todos os times ===
pontuacoes_liga = []

for _, row in df_times.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])
    
    print(f"🔍 Buscando escalação de: {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)
    
    if resp_time.status_code != 200:
        print(f"❌ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    pontuacao_total = 0
    for jogador in atletas:
        atleta_id = jogador['atleta_id']
        if str(atleta_id) in pontuados:
            pontuacao_total += pontuados[str(atleta_id)]['pontuacao']
    
    pontuacoes_liga.append({
        'Nome do Time': nome_time,
        'ID do Time': id_time,
        'Pontuação Parcial': round(pontuacao_total, 2)
    })
    
    # Boa prática: esperar um pouco para não sobrecarregar a API
    sleep(0.5)

# === 4. Monta DataFrame final com pontuação da liga ===
df_resultado = pd.DataFrame(pontuacoes_liga)
df_resultado = df_resultado.sort_values(by='Pontuação Parcial', ascending=False).reset_index(drop=True)

,Nome do Time,ID do Time,Link do Time
0,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598
1,Analove10 ITAQUI GRANDE!!,44574236,https://cartola.globo.com/#!/time/44574236
2,BORGES ITAQUI F.C.,3914981,https://cartola.globo.com/#!/time/3914981
3,cartola scheuer,3851966,https://cartola.globo.com/#!/time/3851966
4,DM Studio,387186,https://cartola.globo.com/#!/time/387186


🔄 Consultando pontuações parciais...


Exception: ❌ Erro ao buscar pontuações parciais.

In [15]:
# Exibe o resultado
display(df_resultado.head(30))

NameError: name 'df_resultado' is not defined

In [34]:
import pandas as pd
import requests
import time
from time import sleep

# === 1. Lê o arquivo Excel ===
arquivo = "links_times_cartola_liga_serie_A.xlsx"
df_times = pd.read_excel(arquivo)

# Verifica colunas
colunas_esperadas = ['Nome do Time', 'ID do Time']
if not all(col in df_times.columns for col in colunas_esperadas):
    raise ValueError(f"O arquivo deve conter as colunas: {colunas_esperadas}")

# === 2. FILTRO: adicione os nomes desejados aqui ===
# Deixe a lista vazia para consultar todos

times_desejados = [
    "FBC Colorado",    
    "Gig@ntte",
    "E.C. Bororé",
    "Real SCI",    
    "GaúchoDaFronteira F.C",
    "OlhaEleAiF.C!",
    "KING LEONN",
    "Texas Club 2025",
    "FBC Colorado II",
    "FBC Colorado III"
]

# times_desejados = ["EdshooowFC", "Defenders F. C.", "Riacho s6", "VIVOPELAGRACA", "Barbosinha FC", "FairPlay Tiro Curto 3",
#                "28 di Marco EC", "QSITO", "jhullyacesar@", "PRECHEKOS FC", "FBC Colorado"]

# Aplica o filtro (ignora maiúsculas/minúsculas)
if times_desejados:
    df_times = df_times[df_times['Nome do Time'].str.lower().isin([t.lower() for t in times_desejados])]
    if df_times.empty:
        raise ValueError("❌ Nenhum dos times filtrados foi encontrado no arquivo.")

# === 3. Consulta pontuações parciais ===
print("🔄 Buscando pontuações parciais...")
url_parciais = "https://api.cartola.globo.com/atletas/pontuados"
resp_parciais = requests.get(url_parciais)
if resp_parciais.status_code != 200:
    raise Exception("❌ Erro ao buscar pontuações parciais.")

data_parciais = resp_parciais.json()
pontuados = data_parciais.get("atletas", {})
clubes = data_parciais.get("clubes", {})
posicoes = data_parciais.get("posicoes", {})

# === 4. Loop pelos times selecionados ===
pontuacoes_liga = []

for _, row in df_times.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])

    print(f"🔍 {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)

    if resp_time.status_code != 200:
        print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    pontuacao_total = 0
    for jogador in atletas:
        atleta_id = jogador['atleta_id']
        if str(atleta_id) in pontuados:
            pontuacao_total += pontuados[str(atleta_id)]['pontuacao']

    pontuacoes_liga.append({
        'Nome do Time': nome_time,
        'ID do Time': id_time,
        'Pontuação Parcial': round(pontuacao_total, 2)
    })

    sleep(0.5)

# === 5. Mostra resultado ordenado ===
df_resultado = pd.DataFrame(pontuacoes_liga)
df_resultado = df_resultado.sort_values(by='Pontuação Parcial', ascending=False).reset_index(drop=True)

# Exibe o resultado
display(df_resultado)

🔄 Buscando pontuações parciais...


Exception: ❌ Erro ao buscar pontuações parciais.

In [35]:
import pandas as pd
import requests

# === 1. Lê o arquivo Excel ===
arquivo = "links_times_cartola_liga_serie_A.xlsx"
df_times = pd.read_excel(arquivo)

# Verifica colunas
colunas_esperadas = ['Nome do Time', 'ID do Time']
if not all(col in df_times.columns for col in colunas_esperadas):
    raise ValueError(f"O arquivo deve conter as colunas: {colunas_esperadas}")

# === 2. Defina os 3 times que deseja analisar ===
times_desejados = [
    "FBC Colorado",    
    "Gig@ntte",
    "E.C. Bororé",
    "Real SCI",    
    "GaúchoDaFronteira F.C",
    "OlhaEleAiF.C!",
    "KING LEONN",
    "Texas Club 2025",
    "FBC Colorado II",
    "FBC Colorado III"
]

# times_desejados = ["EdshooowFC", "Defenders F. C.", "Riacho s6", "VIVOPELAGRACA", "Barbosinha FC", "FairPlay Tiro Curto 3",
#                "28 di Marco EC", "QSITO", "jhullyacesar@", "PRECHEKOS FC", "FBC Colorado"]


# Aplica o filtro ignorando maiúsculas/minúsculas
df_selecionados = df_times[df_times['Nome do Time'].str.lower().isin([t.lower() for t in times_desejados])]
if df_selecionados.empty:
    raise ValueError("❌ Nenhum dos times desejados foi encontrado.")

# === 3. Consulta pontuações parciais ===
print("🔄 Buscando pontuações parciais...")
url_parciais = "https://api.cartola.globo.com/atletas/pontuados"
resp_parciais = requests.get(url_parciais)
if resp_parciais.status_code != 200:
    raise Exception("❌ Erro ao buscar pontuações parciais.")

data_parciais = resp_parciais.json()
pontuados = data_parciais.get("atletas", {})
clubes = data_parciais.get("clubes", {})
posicoes = data_parciais.get("posicoes", {})

# === 4. Exibe os jogadores escalados e as pontuações parciais por time ===
for _, row in df_selecionados.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])

    print(f"\n🔍 {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)

    if resp_time.status_code != 200:
        print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    lista_jogadores = []
    for jogador in atletas:
        atleta_id = jogador['atleta_id']
        if str(atleta_id) in pontuados:
            info = pontuados[str(atleta_id)]
            nome = info['apelido']
            pontuacao = info['pontuacao']
            clube = clubes[str(info['clube_id'])]['nome']
            posicao = posicoes[str(info['posicao_id'])]['nome']
            lista_jogadores.append({
                'Nome do Jogador': nome,
                'Clube': clube,
                'Posição': posicao,
                'Pontuação': pontuacao
            })

    df_jogadores = pd.DataFrame(lista_jogadores).sort_values(by='Pontuação', ascending=False).reset_index(drop=True)
    total = df_jogadores['Pontuação'].sum()

    print(df_jogadores)
    print(f"➡️ Total do time {nome_time}: {total:.2f} pontos")


🔄 Buscando pontuações parciais...


Exception: ❌ Erro ao buscar pontuações parciais.

### Escalações Completas dos Times Escolhidos na Rodada

In [36]:
# Ordem desejada das posições
ordem_posicoes = {
    'Goleiro': 1,
    'Lateral': 2,
    'Zagueiro': 3,
    'Meia': 4,
    'Atacante': 5,
    'Técnico': 6
}

# === 4. Exibe os jogadores escalados e as pontuações parciais por time ===
for _, row in df_selecionados.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])

    print(f"\n🔍 {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)

    if resp_time.status_code != 200:
        print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    lista_jogadores = []
    for jogador in atletas:
        atleta_id = str(jogador['atleta_id'])
        nome = jogador.get('apelido', 'Desconhecido')
        posicao_id = str(jogador.get('posicao_id', '0'))
        clube_id = str(jogador.get('clube_id', '0'))

        pontuacao = pontuados.get(atleta_id, {}).get("pontuacao", 0.0)
        clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")
        posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")

        lista_jogadores.append({
            'Nome do Jogador': nome,
            'Clube': clube,
            'Posição': posicao,
            'Pontuação': pontuacao,
            'Ordem Posição': ordem_posicoes.get(posicao, 99)  # valor alto para posições desconhecidas
        })

    # Ordena por ordem da posição e, em seguida, por pontuação (descrescente)
    df_jogadores = pd.DataFrame(lista_jogadores).sort_values(
        by=['Ordem Posição', 'Pontuação'], ascending=[True, False]
    ).reset_index(drop=True)

    df_jogadores.drop(columns=['Ordem Posição'], inplace=True)

    total = df_jogadores['Pontuação'].sum()

    print(df_jogadores)
    print(f"➡️ Total do time {nome_time}: {total:.2f} pontos")



🔍 FBC Colorado (ID: 186283)


NameError: name 'pontuados' is not defined

### Jogadores mais Escalados por Posição dos Times Selecionados

In [37]:
from collections import defaultdict, Counter

def jogadores_mais_escalados_por_posicao(df_times, pontuados, clubes, posicoes):
    """
    Recebe o DataFrame com times, dados de pontuação parcial, clubes e posições.
    Retorna os jogadores mais escalados por posição.
    """
    escalações = []

    for _, row in df_times.iterrows():
        nome_time = row['Nome do Time']
        id_time = int(row['ID do Time'])

        url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
        resp_time = requests.get(url_escalacao)

        if resp_time.status_code != 200:
            print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
            continue

        dados_time = resp_time.json()
        atletas = dados_time.get("atletas", [])

        for jogador in atletas:
            atleta_id = str(jogador['atleta_id'])
            nome = jogador.get('apelido', 'Desconhecido')
            posicao_id = str(jogador.get('posicao_id', '0'))
            clube_id = str(jogador.get('clube_id', '0'))

            posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")
            clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")

            escalações.append((nome, posicao, clube, atleta_id))

    # Contar jogadores por posição
    contagem = defaultdict(Counter)
    for nome, posicao, clube, atleta_id in escalações:
        chave = f"{nome} ({clube})"
        contagem[posicao][chave] += 1

    # Gerar DataFrames por posição
    dataframes_posicoes = {}
    for posicao, counter in contagem.items():
        df = pd.DataFrame(counter.items(), columns=["Jogador", "Escalações"]).sort_values(by="Escalações", ascending=False)
        dataframes_posicoes[posicao] = df

    return dataframes_posicoes


In [38]:
# Após buscar os dados das pontuações parciais
dataframes_por_posicao = jogadores_mais_escalados_por_posicao(df_selecionados, pontuados, clubes, posicoes)

# Exibir resultados
for posicao in ['Goleiro', 'Lateral', 'Zagueiro', 'Meia', 'Atacante', 'Técnico']:
    print(f"\n📌 {posicao}s mais escalados:")
    if posicao in dataframes_por_posicao:
        print(dataframes_por_posicao[posicao])
    else:
        print("Nenhum jogador escalado nessa posição.")


NameError: name 'pontuados' is not defined

### Jogadores com Scouts

In [39]:
# === 4. Exibe os jogadores escalados e as pontuações parciais por time ===
for _, row in df_selecionados.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])

    print(f"\n🔍 {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)

    if resp_time.status_code != 200:
        print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    lista_jogadores = []
    scouts_unicos = set()

    # Primeiro, coletamos todos os scouts únicos
    for jogador in atletas:
        atleta_id = str(jogador['atleta_id'])
        scout = pontuados.get(atleta_id, {}).get("scout") or {}
        scouts_unicos.update(scout.keys())

    for jogador in atletas:
        atleta_id = str(jogador['atleta_id'])
        nome = jogador.get('apelido', 'Desconhecido')
        posicao_id = str(jogador.get('posicao_id', '0'))
        clube_id = str(jogador.get('clube_id', '0'))

        pontuacao = pontuados.get(atleta_id, {}).get("pontuacao", 0.0)
        clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")
        posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")
        scout = pontuados.get(atleta_id, {}).get("scout") or {}  # ← Aqui está a correção

        jogador_data = {
            'Nome do Jogador': nome,
            'Clube': clube,
            'Posição': posicao,
            'Pontuação': pontuacao
        }

        for s in scouts_unicos:
            jogador_data[s] = scout.get(s, 0)

        lista_jogadores.append(jogador_data)


    df_jogadores = pd.DataFrame(lista_jogadores)

    # Ordena por posição e pontuação
    ordem_posicoes = {
        'Goleiro': 1,
        'Lateral': 2,
        'Zagueiro': 3,
        'Meia': 4,
        'Atacante': 5,
        'Técnico': 6
    }
    df_jogadores['Ordem Posição'] = df_jogadores['Posição'].map(ordem_posicoes)
    df_jogadores = df_jogadores.sort_values(by=['Ordem Posição', 'Pontuação'], ascending=[True, False]).reset_index(drop=True)
    df_jogadores.drop(columns=['Ordem Posição'], inplace=True)

    total = df_jogadores['Pontuação'].sum()

    print(df_jogadores)
    print(f"➡️ Total do time {nome_time}: {total:.2f} pontos")



🔍 FBC Colorado (ID: 186283)


NameError: name 'pontuados' is not defined

### Análise do Scout de todos os jogadores do Cartola

In [40]:
from collections import defaultdict
import pandas as pd
import requests
import time

def coletar_scouts_totais_geral(rodada_final=38):
    scouts_agrupados = defaultdict(lambda: defaultdict(int))
    info_jogadores = {}
    todos_scouts = set()

    for rodada in range(1, rodada_final + 1):
        print(f"🔄 Coletando scouts da rodada {rodada}")
        url = f"https://api.cartola.globo.com/atletas/pontuados/{rodada}"
        resp = requests.get(url)

        if resp.status_code != 200:
            print(f"⚠️ Erro na rodada {rodada}, pulando...")
            continue

        data = resp.json()
        atletas = data.get("atletas", {})
        clubes = data.get("clubes", {})
        posicoes = data.get("posicoes", {})

        for atleta_id, dados in atletas.items():
            nome = dados.get('apelido', 'Desconhecido')
            clube_id = str(dados.get('clube_id', '0'))
            posicao_id = str(dados.get('posicao_id', '0'))
            scout = dados.get("scout") or {}

            clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")
            posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")

            chave_jogador = f"{atleta_id}"

            # Salva dados básicos
            info_jogadores[chave_jogador] = {
                'Nome do Jogador': nome,
                'Clube': clube,
                'Posição': posicao
            }

            for s, valor in scout.items():
                scouts_agrupados[chave_jogador][s] += valor
                todos_scouts.add(s)

        time.sleep(0.4)  # evitar sobrecarga na API

    # Monta o DataFrame final
    jogadores_final = []
    for atleta_id, dados_base in info_jogadores.items():
        linha = dados_base.copy()
        for s in todos_scouts:
            linha[s] = scouts_agrupados[atleta_id].get(s, 0)
        jogadores_final.append(linha)

    df = pd.DataFrame(jogadores_final)

    # Ordena por posição e nome
    ordem_posicoes = {
        'Goleiro': 1,
        'Lateral': 2,
        'Zagueiro': 3,
        'Meia': 4,
        'Atacante': 5,
        'Técnico': 6
    }
    df['Ordem Posição'] = df['Posição'].map(ordem_posicoes)
    df = df.sort_values(by=['Ordem Posição', 'Nome do Jogador']).drop(columns=['Ordem Posição'])

    return df


In [41]:
df_scouts_geral = coletar_scouts_totais_geral(rodada_final=3)


🔄 Coletando scouts da rodada 1
🔄 Coletando scouts da rodada 2
🔄 Coletando scouts da rodada 3


In [42]:
for pos in df_scouts_geral['Posição'].unique():
    print(f"\n📌 {pos}s:")
    display(df_scouts_geral[df_scouts_geral['Posição'] == pos])



📌 Goleiros:


,Nome do Jogador,Clube,Posição,DP,GC,FC,I,PP,SG,CV,PC,FT,DE,G,FF,CA,FS,DS,FD,PS,V,A,GS
199,Alex Muralha,MIR,Goleiro,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,4
95,Anthoni,INT,Goleiro,0,0,0,0,0,2,0,0,0,4,0,0,0,0,0,0,0,0,0,1
384,Brenno,FOR,Goleiro,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
200,Bruno Ferreira,CEA,Goleiro,0,0,1,0,0,1,0,0,0,10,0,0,1,1,0,0,0,0,0,4
264,Caíque França,SPT,Goleiro,0,0,0,0,0,1,0,0,0,8,0,0,0,0,0,0,0,0,0,5
280,Cleiton,RBB,Goleiro,0,0,0,0,0,1,0,0,0,9,0,0,1,0,0,0,0,0,0,4
162,Cássio,CRU,Goleiro,1,0,0,0,0,0,0,0,0,11,0,0,2,0,0,0,0,0,0,5
189,Everson,CAM,Goleiro,0,0,0,0,0,1,0,0,0,9,0,0,0,0,0,0,0,0,0,4
157,Fábio,FLU,Goleiro,0,0,0,0,0,1,0,0,0,7,0,0,0,0,0,0,0,0,0,3
179,Gabriel,SPT,Goleiro,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,2



📌 Laterals:


,Nome do Jogador,Clube,Posição,DP,GC,FC,I,PP,SG,CV,PC,FT,DE,G,FF,CA,FS,DS,FD,PS,V,A,GS
86,Aguirre,INT,Lateral,0,0,4,0,0,2,0,0,0,0,0,0,0,0,6,0,0,0,0,0
168,Alan Ruschel,JUV,Lateral,0,0,1,1,0,1,0,0,0,0,0,0,0,1,4,0,0,0,0,0
167,Alex Sandro,FLA,Lateral,0,0,9,0,0,1,0,0,0,0,0,1,0,4,7,0,0,0,0,0
191,Alex Telles,BOT,Lateral,0,0,8,0,0,2,0,0,0,0,0,1,0,3,2,1,0,0,0,0
88,Andrés Hurtado,RBB,Lateral,0,0,4,0,0,1,0,0,0,0,0,2,0,2,4,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,Vanderlan,PAL,Lateral,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Vitinho,BOT,Lateral,0,0,1,0,0,2,0,0,0,0,0,0,0,0,5,0,0,0,0,0
208,Wendell,SAO,Lateral,0,0,3,0,0,2,0,0,0,0,0,0,0,6,2,0,0,0,0,0
106,Wesley,FLA,Lateral,0,0,6,1,0,1,0,0,0,0,0,0,0,4,5,0,0,0,0,0



📌 Zagueiros:


...

,Nome do Jogador,Clube,Posição,DP,GC,FC,I,PP,SG,CV,PC,FT,DE,G,FF,CA,FS,DS,FD,PS,V,A,GS
45,Acevedo,BAH,Meia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
94,Adson,VAS,Meia,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
160,Alan Franco,CAM,Meia,0,0,2,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0
170,Alan Patrick,INT,Meia,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0
229,Alex Santana,GRE,Meia,0,0,2,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Villasanti,GRE,Meia,0,0,6,0,0,0,0,0,0,0,0,1,0,1,10,0,0,0,0,0
248,Walace,CRU,Meia,0,0,2,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
215,Willian Oliveira,VIT,Meia,0,0,2,0,0,0,0,0,0,0,0,1,1,5,1,0,0,0,0,0
205,Yago Pikachu,FOR,Meia,0,0,1,3,0,0,0,0,0,0,0,1,0,2,5,1,0,0,0,0



📌 Atacantes:


,Nome do Jogador,Clube,Posição,DP,GC,FC,I,PP,SG,CV,PC,FT,DE,G,FF,CA,FS,DS,FD,PS,V,A,GS
8,Ademir,BAH,Atacante,0,0,3,0,0,0,0,0,0,0,0,1,0,6,2,1,0,0,0,0
357,Alex Teixeira,VAS,Atacante,0,0,2,0,0,0,0,0,0,0,0,0,1,0,2,1,0,0,0,0
332,Allanzinho,FOR,Atacante,0,0,2,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0
398,Alysson,GRE,Atacante,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,1,0,0,0,0
154,Amuzu,GRE,Atacante,0,0,2,2,0,0,0,0,0,0,0,2,1,5,1,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,Wesley,INT,Atacante,0,0,2,0,0,0,0,0,0,0,0,2,0,7,6,2,0,0,1,0
360,Willian José,BAH,Atacante,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,1,0,0,0,0
0,Yuri Alberto,COR,Atacante,0,0,3,1,0,0,0,0,0,0,1,1,1,3,0,2,0,0,1,0
246,Ángel Romero,COR,Atacante,0,0,2,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0



📌 Técnicos:


,Nome do Jogador,Clube,Posição,DP,GC,FC,I,PP,SG,CV,PC,FT,DE,G,FF,CA,FS,DS,FD,PS,V,A,GS
85,Abel Ferreira,PAL,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
163,Cuca,CAM,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
401,César Sampaio,SAN,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
135,Fernando Seabra,RBB,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
144,Filipe Luís,FLA,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
182,Fábio Carille,VAS,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
73,Fábio Matias,JUV,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
192,Gustavo Quinteros,GRE,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
43,Juan Vojvoda,FOR,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
249,Leonardo Jardim,CRU,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [43]:
import pandas as pd
from collections import defaultdict
import requests
import time
import json
import os

# Função para coletar os scouts totais de todos os jogadores do Cartola
def coletar_scouts_totais_geral(rodada_final=3):
    scouts_agrupados = defaultdict(lambda: defaultdict(int))
    info_jogadores = {}
    todos_scouts = set()

    for rodada in range(1, rodada_final + 1):
        url = f"https://api.cartola.globo.com/atletas/pontuados/{rodada}"
        resp = requests.get(url)
        if resp.status_code != 200:
            continue

        data = resp.json()
        atletas = data.get("atletas", {})
        clubes = data.get("clubes", {})
        posicoes = data.get("posicoes", {})

        for atleta_id, dados in atletas.items():
            nome = dados.get('apelido', 'Desconhecido')
            clube_id = str(dados.get('clube_id', '0'))
            posicao_id = str(dados.get('posicao_id', '0'))
            scout = dados.get("scout") or {}

            clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")
            posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")

            chave_jogador = f"{atleta_id}"

            info_jogadores[chave_jogador] = {
                'Nome do Jogador': nome,
                'Clube': clube,
                'Posição': posicao
            }

            for s, valor in scout.items():
                scouts_agrupados[chave_jogador][s] += valor
                todos_scouts.add(s)

        time.sleep(0.4)

    jogadores_final = []
    for atleta_id, dados_base in info_jogadores.items():
        linha = dados_base.copy()
        for s in todos_scouts:
            linha[s] = scouts_agrupados[atleta_id].get(s, 0)
        jogadores_final.append(linha)

    df = pd.DataFrame(jogadores_final)

    ordem_posicoes = {
        'Goleiro': 1,
        'Lateral': 2,
        'Zagueiro': 3,
        'Meia': 4,
        'Atacante': 5,
        'Técnico': 6
    }
    df['Ordem Posição'] = df['Posição'].map(ordem_posicoes)
    df = df.sort_values(by=['Ordem Posição', 'Nome do Jogador']).drop(columns=['Ordem Posição'])

    return df

# Coleta os dados
df_scouts_geral = coletar_scouts_totais_geral(rodada_final=3)

# Exporta para Excel com abas por posição
excel_path = "scouts_totais_por_posicao.xlsx"
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    for posicao, grupo in df_scouts_geral.groupby("Posição"):
        grupo.to_excel(writer, sheet_name=posicao[:31], index=False)

# Exporta para arquivo .js
# Agrupa por posição e transforma em dicionário
js_dict = {
    posicao: grupo.dropna(axis=1, how='all').to_dict(orient="records")
    for posicao, grupo in df_scouts_geral.groupby("Posição")
}

# Salva como um arquivo .js válido
js_path = "scouts_totais.js"
with open(js_path, "w", encoding="utf-8") as f:
    f.write("const scoutsTotais = ")
    json.dump(js_dict, f, ensure_ascii=False, indent=2)
    f.write(";")


excel_path, js_path


('scouts_totais_por_posicao.xlsx', 'scouts_totais.js')